# Botanical research - Plant seedlings classification

### Importing and understanding the data

In [1]:
from zipfile import ZipFile
import cv2
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd

In [2]:
images = 'plant-seedlings-classification.zip'

In [3]:
with ZipFile(images,'r') as zip:
    zip.extractall()

In [4]:
path = 'C:/Users/arunv/Downloads/plant-seedlings-classification/train/*/*.png'

In [ ]:
files = glob(path)

In [ ]:
files

In [ ]:
image = []
names = []
j = 1
for img in files:
    src = cv2.imread(img)
    images = cv2.cvtColor(src,cv2.COLOR_BGR2RGB)
    images = cv2.resize(images,(70,70))
    image.append(images)
    names.append(img.split('\\')[-2])
    j += 1

In [ ]:
df = pd.DataFrame(data=names, columns = ["Names"])

In [ ]:
df

In [ ]:
class_names = df['Names'].unique()

In [ ]:
class_labels = {class_name:i for i,class_name in enumerate(class_names)}
class_labels

In [ ]:
labels = [value for m in df.Names for key, value in class_labels.items() if m == key]

In [ ]:
df['Labels'] = labels

In [ ]:
df.head(5)

In [ ]:
image = np.array(image)

In [ ]:
image.shape

In [ ]:
type(image)

In [ ]:
df['image'] = [image[i] for i in range(image.shape[0])]

In [ ]:
df['image'].shape

In [ ]:
df.head(5)

In [ ]:
import random
n = 1
plt.figure(figsize=(20, 20))
num = int(input())
for i in random.sample(range(0, image.shape[0]), num):
    plt.subplot(num//5 ,5 ,n)
    plt.imshow(image[i])
    n = n+1
plt.show()


### Data Preprocessing

In [ ]:
X = df.image.tolist()

In [ ]:
X = np.array(X)

In [ ]:
X.shape

In [ ]:
y = df['Labels']

In [ ]:
y.shape

In [ ]:
from keras.utils import to_categorical

In [ ]:
y = to_categorical(y)

In [ ]:
y.shape

In [ ]:
X = X/255

In [ ]:
X.shape

### Model Training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2 ,random_state= 42)

In [ ]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test.shape}')

In [ ]:
num_classes = y_train.shape[1]

In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
# CNN architecture

model = Sequential()


model.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(70, 70, 3)))

 
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))


model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(rate=0.2))


model.add(Flatten())

model.add(Dense(128, activation="relu"))

model.add(Dense(12, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=50,  
                                                          width_shift_range=0.1,  
                                                          height_shift_range=0.1,  
                                                          horizontal_flip=True, 
                                                          shear_range=0.2,
                                                          zoom_range=0.2,  
                                                          fill_mode="nearest")  
datagen.fit(X_train)

In [ ]:
hist = model.fit(datagen.flow(X_train, y_train, batch_size=20), epochs=50, validation_data=(X_test, y_test))

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred=[np.argmax(i) for i in y_pred ]
y_test = [np.argmax(i) for i in y_test]

In [ ]:
y_test[1]

In [ ]:
n = 1
a = []
b = []
plt.figure(figsize=(20, 20))
num = int(input())
for i in random.sample(range(0, X_test.shape[0]), num):
    plt.subplot(num//5 ,5 ,n)
    plt.imshow(X_test[i])
    a.append(y_test[i])
    b.append(y_pred[i])
    n = n+1
    
plt.show()
print(f'The corresponding labels are: {a}')
print(f'The predicted labels are: {b}')

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(hist.history['accuracy'],'y',linewidth=3.0)
plt.plot(hist.history['val_accuracy'],'g',linewidth=3.0)
plt.legend(['Training accuracy', 'Validation accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy curves',fontsize=16)

# Project2

## Importing and understanding the data

In [ ]:
from tflearn.datasets import oxflower17
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
X,y = oxflower17.load_data()

In [ ]:
print(f'Number of images is :{X.shape[0]}')
print(f'Shape of images is: {X[0].shape}')

In [ ]:
pd.Series(y).value_counts(ascending = True)

## Image exploration and transformation

In [ ]:
import random
n = 1
b=[]
plt.figure(figsize=(20, 20))
for i in random.sample(range(0, 1360), 5):
    plt.subplot(1 ,5 ,n)
    plt.imshow(X[i])
    n = n+1
    plt.axis('off')
    b.append(y[i])
plt.show()
print(f'Labels of each images are:{b}')

In [ ]:
img = X[20]

In [ ]:
plt.imshow(img[:,:,0], cmap='gray')
plt.show

In [ ]:
from scipy.signal import convolve2d

kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])

sharpen = convolve2d(img[:,:,1], kernel, mode = 'valid')
plt.figure(figsize=(20, 20))
plt.subplot(1, 5, 1)
plt.imshow(sharpen,cmap='gray')
plt.subplot(1, 5, 2)
plt.imshow(img[:,:,1],cmap = 'gray')
plt.show

In [ ]:
kernel = np.array([[1/20, 1/20, 1/20],
                   [1/20, 1/20, 1/20],
                   [1/20, 1/20, 1/20]])

blurred = convolve2d(img[:,:,0], kernel, mode = 'valid')
plt.figure(figsize=(20, 20))
plt.subplot(1, 5, 1)
plt.imshow(blurred,cmap='gray')
plt.show
plt.subplot(1, 5, 2)
plt.imshow(img[:,:,0],cmap = 'gray')
plt.show

In [ ]:
plt.figure(figsize=(20, 20))

plt.subplot(4, 1, 1)
plt.imshow(sharpen,cmap='gray')
plt.axis('off')
plt.subplot(4, 1, 2)
plt.imshow(img[:,:,1],cmap = 'gray')
plt.axis('off')
plt.subplot(4, 1, 3)
plt.imshow(blurred,cmap='gray')
plt.axis('off')
plt.subplot(4, 1, 4)
plt.imshow(img[:,:,0],cmap = 'gray')
plt.axis('off')
plt.show

## Model training and tuning

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)

In [ ]:
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of y_train: {y_train1.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_test: {y_test1.shape}')

In [ ]:
import cv2
def resize(array):
    list1 = []
    for i in range(array.shape[0]):
        images = cv2.resize(array[i],(70,70))
        list1.append(images)
    array1 = np.array(list1)
    return array1

In [ ]:
X_train1 = resize(X_train)
X_test1 = resize(X_test)

In [ ]:
print(f'Shape of X_train: {X_train1.shape}')
print(f'Shape of y_train: {y_train1.shape}')
print(f'Shape of X_test: {X_test1.shape}')
print(f'Shape of y_test: {y_test1.shape}')

In [ ]:
num_classes = y_train1.shape[-1]
num_classes

In [ ]:
def flatten(array):
    list1 = []
    for i in range(array.shape[0]):
        images = cv2.resize(array[i],(70,70))
        images = images.flatten()
        list1.append(images)
    array1 = np.array(list1)
    return array1   

In [ ]:
X_train_new = flatten(X_train)
X_test_new = flatten(X_test)

In [ ]:
print(f'Shape of X_train flaatened: {X_train_new.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of X_test flattened: {X_test_new.shape}')
print(f'Shape of y_test: {y_test.shape}')

### Supervised Learning Model

In [ ]:
from sklearn.svm import SVC
SVM = SVC(C=0.8,kernel='poly')
SVM.fit(X_train_new, y_train)
y_pred_svm = SVM.predict(X_test_new)
SVM_Accuracy=SVM.score(X_test_new, y_test)
print("\nAccuracy using SVM : ", SVM_Accuracy*100)

### Neural Network architecture

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Input

In [ ]:
model = Sequential()  
model.add(Flatten(input_shape=(70,70,3)))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=50,  
                                                          width_shift_range=0.1,  
                                                          height_shift_range=0.1,  
                                                          horizontal_flip=True, 
                                                          shear_range=0.2,
                                                          zoom_range=0.2,  
                                                          fill_mode="nearest")  
datagen.fit(X_train1)

In [ ]:
hist = model.fit(datagen.flow(X_train1, y_train1, batch_size=20), epochs=50, validation_data=(X_test1, y_test1))

In [ ]:
model.evaluate(X_test1, y_test1)

### CNN architecture

In [ ]:
model1 = Sequential()


model1.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(70, 70, 3)))

 
model1.add(Conv2D(filters=64, kernel_size=3, activation="relu"))


model1.add(MaxPooling2D(pool_size=(2, 2)))


model1.add(Dropout(rate=0.2))


model1.add(Flatten())

model1.add(Dense(128, activation="relu"))

model1.add(Dense(num_classes, activation="softmax"))
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

In [ ]:
hist = model1.fit(datagen.flow(X_train1, y_train1, batch_size=10), epochs=50, validation_data=(X_test1, y_test1))

In [ ]:
model1.evaluate(X_test1, y_test1)

In [ ]:
y_pred = model1.predict(X_test1)

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]
y_test1 = [np.argmax(i) for i in y_test1]

In [ ]:
y_pred[1:10]

In [ ]:
y_test1[1:10]

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

img = io.imread('Prediction+-+Flower.jpg')

plt.imshow(img)
plt.show()

In [ ]:
img.shape

In [ ]:
images = cv2.resize(img,(70,70))

In [ ]:
np.argmax(model1.predict(images.reshape(1,70,70,3)))

In [ ]:
plt.imshow(X_test[5])

In [ ]:
y_test[5]